## Read data batch with sscIO, apply corrections (empty, flatfield, mask) and restore data

In [27]:
from sscCdi.caterete.ptycho_restauration import restauration_processing_binning
import json
import os
import numpy as np
import h5py
from sscIO import io
from sscPimega import pi540D

def Geometry(L):
    project = pi540D.get_detector_dictionary( L, {'geo':'nonplanar','opt':True,'mode':'virtual'} ) 
    geo = pi540D.geometry540D( project )
    return geo

def restoration_via_interface(data_path,inputs,apply_flat=True,apply_empty=True,apply_mask=True, save_data=True):

    
    print(os.path.join(data_path.rsplit('/',2)[0],'mdata.json'))
    # Read json 
    metadata = json.load(open(os.path.join(data_path.rsplit('/',2)[0],'mdata.json')))
    empty_path = os.path.join(data_path.rsplit('/',2)[0],'images','empty.hdf5')
    flat_path = os.path.join(data_path.rsplit('/',2)[0],'images','flat.hdf5')
    mask_path = os.path.join(data_path.rsplit('/',2)[0],'images','mask.hdf5')    
    distance = metadata['/entry/beamline/experiment']["distance"]*1e-3 # convert to meters    

    n_of_threads, centerx, centery = inputs

    L = 3072 # PIMEGA540D
    half_square_side = min(min(centerx,L-centerx),min(centery,L-centery)) # get the biggest size possible such that the restored difpad is still squared

    os.system(f"h5clear -s {data_path}") # gambiarra because file is not closed at the backend!
    os.system(f"h5clear -s {empty_path}") # gambiarra because file is not closed at the backend!

    print("Reading raw data...")
    raw_difpads,_ = io.read_volume(data_path, 'numpy', use_MPI=True, nprocs=n_of_threads)

    """ Get inputs from json file """
    z1 = float(distance) * 1000 

    """ How the corrections are made prior to restoration:
    
        img[empty > 1] = -1 # Apply empty 
        img = img * np.squeeze(flat) # Apply flatfield
        img[np.abs(mask) ==1] = -1   # Apply Mask
        img = img[cy - hsize:cy + hsize, cx - hsize:cx + hsize] # Center data

    """
    if apply_empty:
        print("Applying empty...")
        empty = np.asarray(h5py.File(empty_path, 'r')['/entry/data/data']).squeeze().astype(np.float32)
    else:
        empty = np.zeros_like(raw_difpads[0])

    if apply_flat:
        print("Applying flatfield...")
        flat = np.array(h5py.File(flat_path, 'r')['entry/data/data'][()][0, 0, :, :])
        flat[np.isnan(flat)] = -1
        flat[flat == 0] = 1
    else:
        flat = np.ones_like(raw_difpads[0])

    if apply_mask:
        print("Applying mask...")
        mask = h5py.File(mask_path, 'r')['entry/data/data'][()][0, 0, :, :]
    else: 
        mask  = np.zeros_like(raw_difpads[0])

    Binning = 4 # standard is 4 for now

    """ Get detector geometry from distance """
    geometry = Geometry(z1)

    jason = {} # dummy dictionary with dummy values to be used within restoration function 
    jason["DetectorExposure"] = [False,0.15]
    jason["CentralMask"] = [False,5]
    jason["DifpadCenter"] = [centerx, centery]
    
    """ Call corrections and restoration """
    print("Restoring diffraction patterns... ")
    r_params = (Binning, empty, flat, centerx, centery, half_square_side, geometry, mask, jason)
    output, _ = pi540D.backward540D_nonplanar_batch(raw_difpads, z1, n_of_threads, [ half_square_side//2 , half_square_side//2 ], restauration_processing_binning,  r_params, 'only') # Apply empty, flatfield, mask and restore!

    if save_data:
        print("Saving data...")
        savepath = os.path.join(data_path.rsplit('/',5)[0],'proc','recons','restoration',data_path.rsplit('/',2)[-1])
        if not os.path.exists(savepath.rsplit('/',1)[0]):
            os.makedirs(savepath.rsplit('/',1)[0])
        h5f = h5py.File(savepath, 'w')
        h5f.create_dataset(data_path.rsplit('/',2)[-1][:-5], data=output)
        h5f.close()

    print("Done!")

    return output

%matplotlib widget
import ipywidgets as widgets
from ipywidgets import fixed
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.cm
from sscCdi import jupyter


def deploy_frame_viewer(path='/ibira/lnls/labs/tepui/proposals/20210062/yuri/yuri_ssc-cdi/other_scripts/mario.npy',axis=0):

    data = np.load(path)
    
    colornorm=colors.Normalize(vmin=data.min(), vmax=data.max())
    
    def update_imshow(sinogram,figure,subplot,frame_number,top=0, bottom=None,left=0,right=None,axis=0,title=False,clear_axis=True,cmap='gray',norm=colors.Normalize()):
        subplot.clear()
        if bottom == None or right == None:
            if axis == 0:
                subplot.imshow(sinogram[frame_number,top:bottom,left:right],cmap=cmap,norm=norm)
            elif axis == 1:
                subplot.imshow(sinogram[top:bottom,frame_number,left:right],cmap=cmap,norm=norm)
            elif axis == 2:
                subplot.imshow(sinogram[top:bottom,left:right,frame_number],cmap=cmap,norm=norm)
        else:
            if axis == 0:
                subplot.imshow(sinogram[frame_number,top:-bottom,left:-right],cmap=cmap,norm=norm)
            elif axis == 1:
                subplot.imshow(sinogram[top:-bottom,frame_number,left:-right],cmap=cmap,norm=norm)
            elif axis == 2:
                subplot.imshow(sinogram[top:-bottom,left:-right,frame_number],cmap=cmap,norm=norm)
        if title == True:
            subplot.set_title(f'Frame #{frame_number}')
        if clear_axis == True:
            subplot.set_xticks([])
            subplot.set_yticks([])    
        figure.canvas.draw_idle()
    
    output = widgets.Output()
    
    with output:
        figure, ax = plt.subplots(dpi=100)
        ax.imshow(np.random.random((4,4)),cmap='gray')
        figure.canvas.draw_idle()
        figure.canvas.header_visible = False
        figure.colorbar(matplotlib.cm.ScalarMappable(norm=colornorm, cmap='gray'))
        plt.show()   

    play_box, selection_slider,play_control = jupyter.slide_and_play(label="Frame Selector",frame_time_milisec=300)

    selection_slider.widget.max, selection_slider.widget.value = data.shape[0] - 1, data.shape[0]//2
    play_control.widget.max =  selection_slider.widget.max
    widgets.interactive_output(update_imshow, {'sinogram':fixed(data),'figure':fixed(figure),'title':fixed(True),'subplot':fixed(ax),'axis':fixed(axis), 'norm':fixed(colornorm),'frame_number': selection_slider.widget})    
    box = widgets.VBox([play_box,output])
    return box

# deploy_frame_viewer(axis=1)

In [28]:
path.rsplit('/',5)[0]

'/ibira/lnls/beamlines/caterete/apps/gcc-jupyter/00000000'

In [29]:
path = "/ibira/lnls/beamlines/caterete/apps/gcc-jupyter/00000000/data/ptycho2d/SS61/scans/0000_SS61_001.hdf5" # INPUT

inputs = [32,1400,1400] # [n_of_threads, centerx, centery]

processed_data = restoration_via_interface(path,inputs)

/ibira/lnls/beamlines/caterete/apps/gcc-jupyter/00000000/data/ptycho2d/SS61/mdata.json
Reading raw data...
-- Calling MPI code to read /ibira/lnls/beamlines/caterete/apps/gcc-jupyter/00000000/data/ptycho2d/SS61/scans/0000_SS61_001.hdf5
Applying empty...
Applying flatfield...
Applying mask...
Restoring diffraction patterns... 
sscPimega: creating 625x700x700 shared array (SharedArray)
Done: 625 Images corrected within 49.37452030181885 sec
    :    Shared Array creation 0.0036809444427490234 sec
Saving data...
Done!
